In [4]:
!pip install praw

     |████████████████████████████████| 133kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 8.7MB/s eta 0:00:01     |████████████████▍               | 102kB 8.7MB/s eta 0:00:01


In [21]:
# Used to pull data from Reddit
import praw
import pandas as pd
import datetime as dt

In [54]:
reddit = praw.Reddit(client_id='OMnAyXELD9zyJA', 
                     client_secret='fW7tokngC-W9lCtDET4-9n0f5QY', 
                     user_agent='Reddit WebScrapping')

### Data from r/democrats Subreddit

In [55]:
democrats = reddit.subreddit('democrats')

In [83]:
posts = []
# for post in democrats.top(limit = 1):
for post in democrats.hot(limit = 1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created, post.is_self, post.link_flair_text])
dem_posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created', 'is_self', 'link_flair_text'])
created = dem_posts["created"].apply(lambda x: dt.datetime.fromtimestamp(x))
dem_posts['created'] = created

In [82]:
# These are the posts that are self created by the user and contains only text
# For these, we will use 'title' and 'body'
dem_posts[dem_posts['is_self'] == True]

,title,score,id,subreddit,url,num_comments,body,created,is_self,link_flair_text
71,Can we all agree to support whoever the nomine...,39,c6ny8m,democrats,https://www.reddit.com/r/democrats/comments/c6...,101,"I understand that Biden, Haris, or Bernie may ...",1.561747e+09,True,None
74,A conservative disappointed in Democratic Party,0,c766cg,democrats,https://www.reddit.com/r/democrats/comments/c7...,19,"I’m a conservative,I’m probably leaning republ...",1.561851e+09,True,None
88,Democratic Debates - PODCAST - Why can't I fin...,3,c6sgkw,democrats,https://www.reddit.com/r/democrats/comments/c6...,1,I was not able to able to listen to the debate...,1.561770e+09,True,None
98,"Harris last night showed she's a fighter, and ...",9,c6luje,democrats,https://www.reddit.com/r/democrats/comments/c6...,5,Someone who can take down trump and leverage t...,1.561737e+09,True,None
121,I’m losing faith in this country,14,c6f087,democrats,https://www.reddit.com/r/democrats/comments/c6...,28,"Just finished watching the democratic debate, ...",1.561692e+09,True,None
128,Has anyone been swayed to support a different ...,7,c6f27i,democrats,https://www.reddit.com/r/democrats/comments/c6...,42,The first debates are over. Have any of you ch...,1.561692e+09,True,None
129,The next debates will be on July 30 and 31 in ...,5,c6fxz9,democrats,https://www.reddit.com/r/democrats/comments/c6...,1,Mark your calendars.,1.561698e+09,True,None
135,Too many candidates right now. Candidates can’...,7,c6f7k6,democrats,https://www.reddit.com/r/democrats/comments/c6...,5,,1.561693e+09,True,None
136,US Border policy is really putting the “kid” i...,6,c6eqvp,democrats,https://www.reddit.com/r/democrats/comments/c6...,2,,1.561690e+09,True,None
151,After the Debates has it changed who you vote ...,0,c6kxmr,democrats,https://www.reddit.com/r/democrats/comments/c6...,1,So right now plan on voting for wither Represe...,1.561733e+09,True,None


In [84]:
# These are the posts that are simply a posted link and do not have body text
# For these, we will use 'title'
dem_posts[dem_posts['is_self'] == False]

,title,score,id,subreddit,url,num_comments,body,created,is_self,link_flair_text
0,"Lindsey Graham challenger raises $270,000 with...",790,c7er7g,democrats,https://www.salon.com/2019/05/31/lindsey-graha...,33,,2019-06-30 22:26:13,False,None
1,Federal judge: Let doctors into child migrant ...,29,c7ld8f,democrats,https://www.cnn.com/2019/06/30/us/child-migran...,1,,2019-07-01 07:48:11,False,None
2,Trump 2020 campaign secretly working with form...,23,c7m5c8,democrats,https://www.chicagotribune.com/nation-world/ct...,0,,2019-07-01 09:07:27,False,article
3,"Trump sucks up to Putin, embarrassing us yet a...",54,c7ih5f,democrats,https://www.washingtonpost.com/opinions/2019/0...,7,,2019-07-01 03:29:52,False,article
4,"David Hoffman on Twitter: ""Russian journalists...",772,c7948n,democrats,https://twitter.com/atDavidHoffman/status/1144...,25,,2019-06-30 12:36:26,False,None
5,Tucker Carlson: Leading a Country ‘Means Killi...,10,c7knwe,democrats,https://www.thedailybeast.com/tucker-carlson-l...,4,,2019-07-01 06:41:21,False,None
6,New Trump policies could end in deportations f...,14,c7i86g,democrats,https://www.militarytimes.com/news/your-milita...,5,,2019-07-01 03:08:05,False,None
7,Illinois governor signs order to protect trans...,3,c7muv2,democrats,https://www.cnn.com/2019/06/30/us/illinois-gov...,0,,2019-07-01 10:19:21,False,None
8,"Crossing the DMZ, but on the road to nowhere: ...",3,c7munf,democrats,https://www.washingtonpost.com/opinions/2019/0...,0,,2019-07-01 10:18:40,False,article
9,"Julia Davis : ""#Russia's state TV revels in th...",3,c7mnfz,democrats,https://twitter.com/JuliaDavisNews/status/1144...,1,,2019-07-01 09:58:42,False,None


### Data from r/Republican Subreddit

In [61]:
republicans = reddit.subreddit('Republican')

In [76]:
posts = []
# for post in democrats.top(limit = 1):
for post in republicans.hot(limit = 1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created, post.is_self, post.link_flair_text])
rep_posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created', 'is_self', 'link_flair_text'])
created = rep_posts["created"].apply(lambda x: dt.datetime.fromtimestamp(x))
rep_posts['created'] = created